# Python으로 배우는 외부데이터 수집과 정제(2)

## Crawler

### 기본 웹 크롤러
    - Requests로 웹 페이지를 추출하고, lxml로 웹 페이지 스크래핑 및 sqlite3 DB에 데이터 저장
    - 크롤링 대상은 한빛미디어 사이트의 "새로 나온 책" 목록
    - 전형적인 목록/상세 패턴을 가진 웹 사이트를 기반으로 도서 정보 추출 크롤러 제작
    - 목록 페이지는 제목과 저자 정보
    - 상세 페이지의 출간일, 페이지, ISBN, 물류 코드, 책 소개 등의 정보 중 제목, 가격, 목차 정보를 추출

In [1]:
# 목록 페이지에서 상세 페이지로의 링크 목록을 추출
import requests
import lxml.html
!pip install cssselect

In [2]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('.view_box a'):
    url = a.get('href')
    print(url)

/store/books/look.php?p_code=B3178834938
javascript:;
/store/books/look.php?p_code=B3178834938
/store/books/look.php?p_code=B6084607806
javascript:;
/store/books/look.php?p_code=B6084607806
/store/books/look.php?p_code=B8948111854
javascript:;
/store/books/look.php?p_code=B8948111854
/store/books/look.php?p_code=B8661740335
javascript:;
/store/books/look.php?p_code=B8661740335
/store/books/look.php?p_code=B1382037871
javascript:;
/store/books/look.php?p_code=B1382037871
/store/books/look.php?p_code=B2847674054
javascript:;
/store/books/look.php?p_code=B2847674054
/store/books/look.php?p_code=B4162156367
/store/books/look.php?p_code=B4162156367
/store/books/look.php?p_code=B5156679966
/store/books/look.php?p_code=B5156679966
/store/books/look.php?p_code=B6189263297
/store/books/look.php?p_code=B6189263297
/store/books/look.php?p_code=B4813114979
javascript:;
/store/books/look.php?p_code=B4813114979
/store/books/look.php?p_code=B8806843023
javascript:;
/store/books/look.php?p_code=B88068

In [3]:
# "javascipt"로 시작하는 목록 제거 필요
# 상대 URL을 절대 URL로 변환 필요
import requests
import lxml.html
!pip install cssselect

In [4]:
s = requests.Session()
response = s.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)

In [5]:
# 모든 링크를 절대 URL로 변환
root.make_links_absolute(response.url)

# 목록에서 javascript 제거
for a in root.cssselect('.view_box .book_tit a'):
    url = a.get('href')
    print(url)

https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938
https://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806
https://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854
https://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335
https://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871
https://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054
https://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367
https://www.hanbit.co.kr/store/books/look.php?p_code=B5156679966
https://www.hanbit.co.kr/store/books/look.php?p_code=B6189263297
https://www.hanbit.co.kr/store/books/look.php?p_code=B4813114979
https://www.hanbit.co.kr/store/books/look.php?p_code=B8806843023
https://www.hanbit.co.kr/store/books/look.php?p_code=B2859439220
https://www.hanbit.co.kr/store/books/look.php?p_code=B1996635146
https://www.hanbit.co.kr/store/books/look.php?p_code=B9063949643
https://www.hanbit.co.kr/store/books/look.php?p_code=B7400451696
https://www.hanbit.co.kr/

In [6]:
# URL 목록 추출을 위한 scrape_list_page() 함수 정의
# scrape_list_page() 함수의 반환값은 list처럼 반복 가능한 제너레이터로 구현

def main() :
    """
    크롤러의 메인처리
    """
    # 여러 페이지에서 크롤링을 위해  Session 사용
    session = requests.Session()
    
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    
    # 제너레이터는 list처럼 사용 가능
    for url in urls :
        time.sleep(1)          # 1초간 대기 - 블락을 예방
        print(url)
        print('-'*70)

In [7]:
def scrape_list_page(response) :
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a') :
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

## 상세 페이지 스크래핑
- 개발자 도구로 CSSSelector 확인
    - 타이틀 : .store_product_info_box h3
    - 가격 : .pbr strong
    - 목차 : #tabs_3 .hanbit_edit_view 내부의 p 태그들
- response를 매개변숧 scrape_detail_page()를 호출해서 책의 상세 정보를 추출
- scrape_detail_page() 함수에서는 CSS Selector를 사용해 ㅎ스크래핑
- 제목과 가격은 root.cssselect() 함수로 추출한 리스트의 첫 번쨰 요소에서 문자열을 추출
- 목차는 List Conprehension을 사용해 목차를 리스트로 추출

In [8]:
# 목차에 포함돼 있는 공백을 제거할 수 있는 normalize_space() 함수 정의
# List Comprehension 구문에 조건을 추가해서 빈 문자열을 제거
def scrape_detail_page(response) : 
        """
        상세 페이지의 Response에서 책 정보를 dic로 추출
        """
        root = lxml.html.fromstring(response.content)
        ebook = {
            'url' : response.url,
            'title' : root.cssselect('.store_product_info_box h3')[0].text_content(),
            'price' : root.cssselect('.ppbr strong')[0].text_content(),
            'content' : [normalize_spaces(p.text_content())
                        for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                        if normalize_spaces(p.text_content()) != '']
        }
        return ebook

In [9]:
def normalize_spaces(s) :
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [10]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1)                         # 1초간 대기
        response = session.get(url)           # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)                          # 상세 정보 출력
        break

## 고급 웹 크롤러

### daum 뉴스 목록 첫 페이지

In [11]:
import requests
import lxml.html
import pandas as pd
import sqlite3
from pandas.io import sql
import os

In [12]:
REG_DATE = '20200819'

In [13]:
response = requests.get('https://news.daum.net/breakingnews/digital?regDate={}'.format(REG_DATE))
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li') :
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    print(url, a.text)

https://v.daum.net/v/20200819230943259 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230922257 트럼프 "오라클은 대단한 회사"..틱톡 인수 지지
https://v.daum.net/v/20200819230858252 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230757249 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230752248 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230732247 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230639242 한국심리학회 연차학술대회 20~22일 열려
https://v.daum.net/v/20200819230626241 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230623240 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230557231 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230516226 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230512224 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819230448221 RUSSIA SPACE DOGS BELKA AND STRELKA
https://v.daum.net/v/20200819225027087 글로벌창업사

### daum 뉴스 목록 상세 페이지

In [14]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    return full_body

get_detail('https://news.v.daum.net/v/20200505000102404')

'LG전자 스마트TV 리모컨 클릭 한 번에 왓챠플레이가 연결된다 왓챠와 LG전자는 LG 2019년형 스마트TV부터 스마트TV 리모컨에서 영화 버튼을 클릭하면 왓챠플레이 서비스가 바로 연동되도록 지원한다 왓챠플레이를 구독하는 고객 편의성을 강화했다 LG 신형 스마트TV를 보유한 왓챠플레이 고객은 편리하게 서비스를 이용할 수 있게 됐다 기존에도 삼성전자LG전자 스마트TV 메뉴와 유료방송 셋톱박스 등을 통해 왓챠플레이를 시청할 수 있었다 왓챠플레이 애플리케이션앱을 찾아야 하는 번거로움이 있었다 박태훈 왓챠 대표는 국내외 제조사는 물론 통신사 방송사와 서비스 협업을 논의 중이라며 왓챠플레이를 더 쉽게 접할 수 있도록 계속 협력해나갈 것이라고 말했다 왓챠플레이뿐만 아니라 웨이브 넷플릭스 디즈니플러스디즈니 등 대다수 온라인동영상서비스OTT가 스마트TV유료방송과 고객 접점을 확대하는 추세다 스마트폰태블릿PC 등 모바일은 물론 TV와 PC 등 다양한 디바이스에서 클릭 한 번에 서비스를 이용하도록 지원 접근성을 강화했다 고객 접점을 확대해 가입 유인을 늘리기 위한 전략이다 넷플릭스는 LG유플러스와 LG헬로비전 딜라이브 등 IPTV케이블TV와 서비스를 연동 쉽게 볼 수 있도록 지원한다 삼성LG 스마트TV 앱 또는 리모컨에서 넷플릭스NETFLIX 버튼 클릭 한 번에 서비스를 이용할 수 있도록 지원한다 웨이브와 CJ ENM 티빙TVING 등 국산 OTT는 물론 미국유럽 등 해외에서 서비스를 제공 중인 디즈니 역시 삼성LG전자 스마트TV에서 서비스를 제공한다 유료방송 플랫폼 차원에서 OTT와 제휴도 적극적이다 KT스카이라이프는 통합 OTT 서비스 토핑 출시 1주년을 맞아 국내 최대 애니메이션 스트리밍 플랫폼 라프텔 서비스를 추가했다 이달 초부터 라프텔을 통해 진격의 거인 소드 아트 온라인 도쿄 구울 나루토 원피스 등 인기 애니메이션을 회차별 결제 없이 월 9900원에 볼 수 있다 제공 OTT를 지속 확대할 계획이다 왓챠플레이와 웨이브는 토핑에서 유료 가입시청할 수 있다 유튜브는 무료

# Web Crawlers

In [15]:
import requests
import lxml.html
import re
import time

In [16]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)
for a in root.cssselect('.view_box a'):
    url = a.get('href')
    print(url)

/store/books/look.php?p_code=B3178834938
javascript:;
/store/books/look.php?p_code=B3178834938
/store/books/look.php?p_code=B6084607806
javascript:;
/store/books/look.php?p_code=B6084607806
/store/books/look.php?p_code=B8948111854
javascript:;
/store/books/look.php?p_code=B8948111854
/store/books/look.php?p_code=B8661740335
javascript:;
/store/books/look.php?p_code=B8661740335
/store/books/look.php?p_code=B1382037871
javascript:;
/store/books/look.php?p_code=B1382037871
/store/books/look.php?p_code=B2847674054
javascript:;
/store/books/look.php?p_code=B2847674054
/store/books/look.php?p_code=B4162156367
/store/books/look.php?p_code=B4162156367
/store/books/look.php?p_code=B5156679966
/store/books/look.php?p_code=B5156679966
/store/books/look.php?p_code=B6189263297
/store/books/look.php?p_code=B6189263297
/store/books/look.php?p_code=B4813114979
javascript:;
/store/books/look.php?p_code=B4813114979
/store/books/look.php?p_code=B8806843023
javascript:;
/store/books/look.php?p_code=B88068

In [17]:
response = requests.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
root = lxml.html.fromstring(response.content)

In [18]:
# 모든 링크를 절대 URL로 변환
root.make_links_absolute(response.url)

# 목록에서 javascript 제거
for a in root.cssselect('.view_box .book_tit a'):
    url = a.get('href')
    print(url)

https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938
https://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806
https://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854
https://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335
https://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871
https://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054
https://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367
https://www.hanbit.co.kr/store/books/look.php?p_code=B5156679966
https://www.hanbit.co.kr/store/books/look.php?p_code=B6189263297
https://www.hanbit.co.kr/store/books/look.php?p_code=B4813114979
https://www.hanbit.co.kr/store/books/look.php?p_code=B8806843023
https://www.hanbit.co.kr/store/books/look.php?p_code=B2859439220
https://www.hanbit.co.kr/store/books/look.php?p_code=B1996635146
https://www.hanbit.co.kr/store/books/look.php?p_code=B9063949643
https://www.hanbit.co.kr/store/books/look.php?p_code=B7400451696
https://www.hanbit.co.kr/

In [19]:
def main():
    """
    크롤러의 메인 처리
    """
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    # 제너레이터는 list처럼 사용 가능
    for url in urls:
        print(url)
        print('-'*70)

In [20]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        # yield 구문으로 제너레이터의 요소 반환
        yield url

In [21]:
if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367
----------------------------------------------------------------------
https://www.hanbit.co.kr/store/books/look.php?p_

In [22]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  

In [23]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [24]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [p.text_content()\
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')]
    }
    return ebook

In [25]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['', '\r\n\t\t<1편>\r\n', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '\xa0', '\r\n\t\t<2편>\r\n', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '\xa0', '\r\n\t\t<3편>\r\n', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '\xa0', '\r\n\t\t<4편>\r\n', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}


In [26]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  

In [27]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [28]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [29]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [30]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['<1편>', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '<2편>', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '<3편>', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '<4편>', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}


In [31]:
def main():
    # 여러 페이지에서 크롤링을 위해 Session 사용
    session = requests.Session()  
    # scrape_list_page() 함수를 호출해서 제너레이터를 추출
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1) # 1초간 대기
        response = session.get(url)  # Session을 사용해 상세 페이지를 추출
        ebook = scrape_detail_page(response)  # 상세 페이지에서 상세 정보를 추출
        print(ebook)  # 상세 정보 출력
        break  

In [32]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [33]:
def scrape_detail_page(response):
    """
    상세 페이지의 Response에서 책 정보를 dict로 추출
    """
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url': response.url,
        'title': root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price': root.cssselect('.pbr strong')[0].text_content(),
        'content': [normalize_spaces(p.text_content())
            for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
            if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [34]:
def normalize_spaces(s):
    """
    연결된 공백을 하나의 공백으로 변경
    """
    return re.sub(r'\s+', ' ', s).strip()

In [35]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['<1편>', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '<2편>', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '<3편>', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '<4편>', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}


In [36]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

get_detail('https://news.v.daum.net/v/20200505000102404')

'LG전자 스마트TV 리모컨 클릭 한 번에 왓챠플레이가 연결된다 왓챠와 LG전자는 LG 2019년형 스마트TV부터 스마트TV 리모컨에서 영화 버튼을 클릭하면 왓챠플레이 서비스가 바로 연동되도록 지원한다 왓챠플레이를 구독하는 고객 편의성을 강화했다 LG 신형 스마트TV를 보유한 왓챠플레이 고객은 편리하게 서비스를 이용할 수 있게 됐다 기존에도 삼성전자LG전자 스마트TV 메뉴와 유료방송 셋톱박스 등을 통해 왓챠플레이를 시청할 수 있었다 왓챠플레이 애플리케이션앱을 찾아야 하는 번거로움이 있었다 박태훈 왓챠 대표는 국내외 제조사는 물론 통신사 방송사와 서비스 협업을 논의 중이라며 왓챠플레이를 더 쉽게 접할 수 있도록 계속 협력해나갈 것이라고 말했다 왓챠플레이뿐만 아니라 웨이브 넷플릭스 디즈니플러스디즈니 등 대다수 온라인동영상서비스OTT가 스마트TV유료방송과 고객 접점을 확대하는 추세다 스마트폰태블릿PC 등 모바일은 물론 TV와 PC 등 다양한 디바이스에서 클릭 한 번에 서비스를 이용하도록 지원 접근성을 강화했다 고객 접점을 확대해 가입 유인을 늘리기 위한 전략이다 넷플릭스는 LG유플러스와 LG헬로비전 딜라이브 등 IPTV케이블TV와 서비스를 연동 쉽게 볼 수 있도록 지원한다 삼성LG 스마트TV 앱 또는 리모컨에서 넷플릭스NETFLIX 버튼 클릭 한 번에 서비스를 이용할 수 있도록 지원한다 웨이브와 CJ ENM 티빙TVING 등 국산 OTT는 물론 미국유럽 등 해외에서 서비스를 제공 중인 디즈니 역시 삼성LG전자 스마트TV에서 서비스를 제공한다 유료방송 플랫폼 차원에서 OTT와 제휴도 적극적이다 KT스카이라이프는 통합 OTT 서비스 토핑 출시 1주년을 맞아 국내 최대 애니메이션 스트리밍 플랫폼 라프텔 서비스를 추가했다 이달 초부터 라프텔을 통해 진격의 거인 소드 아트 온라인 도쿄 구울 나루토 원피스 등 인기 애니메이션을 회차별 결제 없이 월 9900원에 볼 수 있다 제공 OTT를 지속 확대할 계획이다 왓챠플레이와 웨이브는 토핑에서 유료 가입시청할 수 있다 유튜브는 무료

In [37]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [38]:
page = 1
max_page = 0
REG_DATE = '20200819'

In [39]:
response = requests.get('http://news.daum.net/breakingnews/digital?page={}&regDate={}'\
                        .format(page, REG_DATE))
root = lxml.html.fromstring(response.content)
for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
    a = li.xpath('div/strong/a')[0]
    url = a.get('href')
    article = get_detail(url)
    print(f'URL : {url}')
    print(f'TITLE : {a.text}')
    print(f'ARTICLE : {article}')
    print('-' * 100)

URL : https://v.daum.net/v/20200819230943259
TITLE : RUSSIA SPACE DOGS BELKA AND STRELKA
ARTICLE : epa08612780 Visitors look at stuffed dogs Belka L and Strelka R at the Museum of Cosmonautics in Moscow Russia 19 August 2020 Russia celebrates the 60th anniversary of KorablSputnik 2 on 19 August The two female dogs Strelka and Belka among other animals the first creatures to return alive from space EPAYURI KOCHETKOV
----------------------------------------------------------------------------------------------------
URL : https://v.daum.net/v/20200819230922257
TITLE : 트럼프 "오라클은 대단한 회사"..틱톡 인수 지지
ARTICLE : 지디넷코리아김익현 미디어연구소장 도널드 트럼프 미국 대통령이 오라클의 틱톡 인수에 강한 지지 의사를 드러냈다 CNBC를 비롯한 주요 외신들에 따르면 트럼프는 19일현지시간 미국 애리조나주 유마 유세 현장에서 오라클이 틱톡을 인수하기에 좋은 기업이냐는 질문을 받고 오라클은 대단한 회사라고 답변했다 트럼프는 오라클이 틱톡 인수에 관심을 갖고 있다는 보도가 나온 직후  이 같은 발언을 했다 트럼프의 예사롭지 않은 발언에 관심이 쏠리는 건 엘리슨의 이런 정치적 성향 때문이다 보도에 따르면 트럼프는 오라클은 대단한 회사이며 오라클 소유주는 엄청난tremendous 사람이라고 생각한다면서 오라클은 틱톡을 잘 다룰 수 있을 것이다고 말했다 트럼프 대통령은 바이트댄스 측에 90일 이내에 틱톡 미국 사업

URL : https://v.daum.net/v/20200819225004085
TITLE : 배틀그라운드, 신규 무기 'MG3'·'교란 수류탄' 추가
ARTICLE : ‘배틀그라운드’가 업데이트를 통해 다양한 신규 무기를 선보인다 두 가지 연사 모드가 가능한 최신 경기관총 ‘MG3’와 신규 투척 무기 ‘교란 수류탄’이 추가됐다 펍지주식회사가 플레이어언노운스 배틀그라운드 82 업데이트를 19일 진행한다 올해 세 번째 배틀그라운드 글로벌 이스포츠 대회인 펍지 콘티넨털 시리즈PUBG Continental Series PCS 2 개막을 앞두고 이스포츠 탭을 업데이트해 다양한 소식과 이벤트를 선보인다 PCS2의 승자를 예측하는 Pick’Em 챌린지 이벤트는 업데이트 이후부터 9월 23일까지 진행된다 Pick’Em 챌린지는 경기 시청 또는 인게임 아이템 구매로 투표권을 획득해 우승팀을 예측하고 성공 시 보상을 받는 이벤트다 PCS2 특별 아이템 구매 및 우승팀 투표는 9월 8일 오후 5시까지 가능하다 예측 성공 시 보상은 이스포츠 탭이 비활성화되는 9월 23일까지 획득할 수 있다 해당 아이템 판매 수익의 25는 4개 권역에 동일하게 배분해 참가팀과 공유된다 이스포츠 탭을 통해 PCS2에 출전한 권역별 최고의 선수 3인 팀별 순위 등도 확인할 수 있다 한편 이번 업데이트로 배틀그라운드에서 다양한 신규 무기를 만나볼 수 있다 보급상자를 통해서만 얻을 수 있는 최신 경기관총 ‘MG3’는 다른 총기와 달리 660rpm분당 발사 수 및 990rpm의 두 가지 연사 모드가 가능하다 MG3에는 양각대가 부착돼 엎드릴 경우 보다 안정적인 사격이 가능하다 또한 다섯 번째 탄환과 총알을 다 소진하기 직전 10발은 예광탄으로 발사되고 차량 사격 시 125배의 대미지를 입힐 수 있는 특징을 갖는다 총기 격발 사운드를 모방하는 신규 투척 무기 ‘교란 수류탄’도 추가됐다 다양한 전술적 도구로 활용이 가능한 교란 수류탄은 사녹에만 소량 스폰된다 투척 후 10초간 가짜 사격음을 만들어 유저의 위치

In [40]:
def db_save(NEWS_LIST):
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con:
        try:
            NEWS_LIST.to_sql(name = 'NEWS_LIST', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
        print(len(NEWS_LIST), '건 저장완료..')

In [41]:
def db_delete():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: 
        try:
            cur = con.cursor()
            sql = 'DELETE FROM NEWS_LIST'
            cur.execute(sql)
        except Exception as e:
            print(str(e)) 

In [42]:
def db_select():
    with sqlite3.connect(os.path.join('.','sqliteDB')) as con: 
        try:
            query = 'SELECT * FROM NEWS_LIST'
            NEWS_LIST = pd.read_sql(query, con = con)
        except Exception as e:
            print(str(e)) 
        return NEWS_LIST

In [43]:
import re
import string

def get_detail(url):
    body = []
    punc = '[!"#$%&\'()*+,-./:;<=>?[\]^_`{|}~“”·]'
    response = requests.get(url)
    root = lxml.html.fromstring(response.content)
    for p in root.xpath('//*[@id="harmonyContainer"]/section/p'):
        if p.text: # 체크
            body.append(re.sub(punc, '', p.text)) # 특수문자 제거
    full_body = ' '.join(body)
    
    return full_body

In [44]:
page = 440
max_page = 0
REG_DATE = '20220718'

In [45]:
while(True):
    df_list = []
    response = requests.get('http://news.daum.net/breakingnews/society?page={}&regDate={}'\
                            .format(page, REG_DATE))
    root = lxml.html.fromstring(response.content)
    for li in root.xpath('//*[@id="mArticle"]/div[3]/ul/li'):
        a = li.xpath('div/strong/a')[0]
        url = a.get('href')
        article = get_detail(url)
        df = pd.DataFrame({'URL' : [url],'TITLE':[a.text],'ARTICLE' : [article]})
        df_list.append(df)   
        
    if df_list:   
        df_10 = pd.concat(df_list)
        db_save(df_10)

    # 페이지 번호 중에서 max 페이지 가져오기    
    for a in root.xpath('//*[@id="mArticle"]/div[3]/div/span/a'):
        try:
            num = int(a.text)
            if max_page < num:
                max_page = num       
        except:
            pass

    # 마지막 페이지 여부 확인     
    span = root.xpath('//*[@id="mArticle"]/div[3]/div/span/a[@class="btn_page btn_next"]')

    if (len(span) <= 0) & (page >= max_page):
        break
    else:
        page = page + 1
        
    time.sleep(1)      

15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
15 건 저장완료..
11 건 저장완료..


In [46]:
# db_delete()

In [47]:
print(db_select())

                                         URL  \
0     https://v.daum.net/v/20200819230943259   
1     https://v.daum.net/v/20200819230922257   
2     https://v.daum.net/v/20200819230858252   
3     https://v.daum.net/v/20200819230757249   
4     https://v.daum.net/v/20200819230752248   
...                                      ...   
1381  https://v.daum.net/v/20220718000239252   
1382  https://v.daum.net/v/20220718000226247   
1383  https://v.daum.net/v/20220718000205246   
1384  https://v.daum.net/v/20220718000145233   
1385  https://v.daum.net/v/20220718000104218   

                                      TITLE  \
0       RUSSIA SPACE DOGS BELKA AND STRELKA   
1               트럼프 "오라클은 대단한 회사"..틱톡 인수 지지   
2       RUSSIA SPACE DOGS BELKA AND STRELKA   
3       RUSSIA SPACE DOGS BELKA AND STRELKA   
4       RUSSIA SPACE DOGS BELKA AND STRELKA   
...                                     ...   
1381  BA.5 BA.2.75 함께 등장, 정점 2번 쌍봉낙타형 유행 우려   
1382           [Hot Poll] '예대금리차 공시' 핫 폴 결과   
